In [ ]:
# Cell 1: 训练模型
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import tempfile
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                             f1_score, roc_auc_score, average_precision_score,
                             roc_curve, precision_recall_curve, confusion_matrix,
                             ConfusionMatrixDisplay, classification_report)
from sklearn.preprocessing import LabelEncoder
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.drawing.image import Image
from joblib import dump, load
import shap

# ====================== 配置参数 ======================
CONFIG = {
    'data_file': "data.xlsx",
    'target': "1yearegfr",
    'categorical_features': ['Crescent-shaped_changes', 'Interstitial_fibrosis'],
    'numerical_features': ['ePWV', 'SII', '24h-UP', 'eGFR'],
    'test_size': 0.25,
    'random_state': 42,
    'base_params': {
        'n_estimators': 500,
        'max_depth': 10,
        'min_samples_split': 10,
        'min_samples_leaf': 5,
        'max_features': 0.1,
        'class_weight': 'balanced',
        'bootstrap': True,
        'oob_score': False,
        'random_state': 42,
        'criterion': 'entropy',
        'n_jobs': -1
    },
    'param_grid': {
        'n_estimators': [300, 500],
        'max_depth': [3, 5, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 3, 5],
        'max_features': ['sqrt', 0.1, 0.5]
    },
    'output_dir': "results\5_RF"
}

# ====================== 工具函数 ======================
def load_and_preprocess_data(file_path):
    """加载并预处理数据"""
    data = pd.read_excel(file_path)
    
    print("数据前5行：")
    print(data.head())
    print("\n数据描述统计：")
    print(data.describe())
    print("\n缺失值检查：")
    print(data.isnull().sum())
    
    return data.dropna()

def preprocess_features(data, categorical_features, numerical_features, target):
    """预处理特征数据"""
    X = data[categorical_features + numerical_features].copy()
    y = data[target]
    
    label_encoder = LabelEncoder()
    for col in categorical_features:
        X.loc[:, col] = label_encoder.fit_transform(X[col])
    
    return X, y

def find_optimal_threshold(y_true, y_prob, method='f1'):
    """自动寻找最佳分类阈值"""
    if method == 'f1':
        precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
        f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
        return thresholds[np.argmax(f1_scores)]
    
    elif method == 'youden':
        fpr, tpr, thresholds = roc_curve(y_true, y_prob)
        return thresholds[np.argmax(tpr - fpr)]
    
    elif method == 'precision_recall':
        precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
        return thresholds[np.argmin(np.abs(precision - recall))]

def evaluate_by_ckd_group(model, X, y_true, ckd_groups, threshold=0.5):
    """按CKD分组评估模型性能"""
    group_metrics = {}
    y_prob = model.predict_proba(X)[:, 1]
    y_pred = (y_prob >= threshold).astype(int)
    
    for group in sorted(ckd_groups.unique()):
        group_indices = ckd_groups == group
        if sum(group_indices) == 0:
            continue
            
        group_y_true = y_true[group_indices]
        group_y_prob = y_prob[group_indices]
        group_y_pred = y_pred[group_indices]
        
        metrics = {
            'n_samples': sum(group_indices),
            'accuracy': accuracy_score(group_y_true, group_y_pred),
            'precision': precision_score(group_y_true, group_y_pred, zero_division=0),
            'recall': recall_score(group_y_true, group_y_pred, zero_division=0),
            'f1': f1_score(group_y_true, group_y_pred, zero_division=0),
            'roc_auc': roc_auc_score(group_y_true, group_y_prob) if len(np.unique(group_y_true)) > 1 else np.nan,
            'pr_auc': average_precision_score(group_y_true, group_y_prob)
        }
        group_metrics[group] = metrics
    
    return group_metrics

def calculate_metrics(y_true, y_pred, y_prob, prefix=''):
    """计算评估指标"""
    return {
        f'{prefix}accuracy': accuracy_score(y_true, y_pred),
        f'{prefix}precision': precision_score(y_true, y_pred, average='binary', zero_division=0),
        f'{prefix}recall': recall_score(y_true, y_pred, average='binary', zero_division=0),
        f'{prefix}f1': f1_score(y_true, y_pred, average='binary', zero_division=0),
        f'{prefix}roc_auc': roc_auc_score(y_true, y_prob),
        f'{prefix}pr_auc': average_precision_score(y_true, y_prob)
    }

def save_individual_plots(best_rf, X_test, y_test, y_prob, X_train, y_train, y_train_prob, 
                         optimal_threshold_f1, optimal_threshold_youden, test_roc_auc, test_pr_auc):
    """保存各个子图"""
    os.makedirs(CONFIG['output_dir'], exist_ok=True)
    
    # 特征重要性
    feature_importance = pd.Series(best_rf.feature_importances_, index=X_train.columns)
    plt.figure(figsize=(6, 4))
    feature_importance.sort_values().plot(kind='barh')
    plt.title('Feature Importance(Gini)')
    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/01_feature_importance.png", dpi=300)
    plt.close()

    # ROC曲线
    fpr_test, tpr_test, _ = roc_curve(y_test, y_prob)
    fpr_train, tpr_train, _ = roc_curve(y_train, y_train_prob)
    train_roc_auc = roc_auc_score(y_train, y_train_prob)
    
    plt.figure(figsize=(6, 4))
    plt.plot(fpr_test, tpr_test, label=f'Test ROC (AUC = {test_roc_auc:.2f})')
    plt.plot(fpr_train, tpr_train, label=f'Train ROC (AUC = {train_roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], 'k--', lw=1)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve (Train vs Test)')
    plt.legend(loc='lower right')
    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/02_roc_curve.png", dpi=300)
    plt.close()

    # PR曲线
    precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
    plt.figure(figsize=(6, 4))
    plt.plot(recall, precision, label=f'Test PR (AUC = {test_pr_auc:.2f})')
    f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
    opt_idx = np.argmax(f1_scores)
    plt.scatter(recall[opt_idx], precision[opt_idx], color='red', label=f'Optimal F1 = {thresholds[opt_idx]:.2f}')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/03_pr_curve.png", dpi=300)
    plt.close()

    # 混淆矩阵
    for threshold, suffix in zip([0.5, optimal_threshold_f1], ['default', 'optimal']):
        y_pred = (y_prob >= threshold).astype(int)
        cm = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(5, 4))
        ConfusionMatrixDisplay(confusion_matrix=cm).plot(cmap='Blues' if suffix == 'default' else 'Greens', colorbar=False)
        plt.title(f'Test CM ({suffix.capitalize()} Threshold={threshold:.2f})')
        plt.tight_layout()
        plt.savefig(f"{CONFIG['output_dir']}/04_cm_{suffix}.png", dpi=300)
        plt.close()

    # 阈值选择
    thresholds_arr = np.linspace(0, 1, 100)
    f1_scores_curve = [f1_score(y_test, (y_prob >= t).astype(int)) for t in thresholds_arr]
    youden_j_curve = [recall_score(y_test, (y_prob >= t).astype(int)) - 
                     (1 - precision_score(y_test, (y_prob >= t).astype(int))) for t in thresholds_arr]
    
    plt.figure(figsize=(6, 4))
    plt.plot(thresholds_arr, f1_scores_curve, label='F1 Score')
    plt.plot(thresholds_arr, youden_j_curve, label="Youden's J")
    plt.axvline(optimal_threshold_f1, color='r', ls='--', label=f'Optimal F1 = {optimal_threshold_f1:.2f}')
    plt.axvline(optimal_threshold_youden, color='g', ls='--', label=f'Optimal Youden = {optimal_threshold_youden:.2f}')
    plt.xlabel('Threshold')
    plt.ylabel('Score')
    plt.title('Threshold Selection')
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/05_threshold_selection.png", dpi=300)
    plt.close()


# ====================== 主执行流程 ======================
def main():
    # 1. 数据准备
    data = load_and_preprocess_data(CONFIG['data_file'])
    X, y = preprocess_features(data, CONFIG['categorical_features'], 
                              CONFIG['numerical_features'], CONFIG['target'])
    
    # 2. 划分数据集
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=CONFIG['test_size'], random_state=CONFIG['random_state'], stratify=y
    )
    
    # 3. 交叉验证
    rf = RandomForestClassifier(**CONFIG['base_params'])
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=CONFIG['random_state'])
    cv_scores = cross_val_score(rf, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)
    
    print(f"\n交叉验证结果（ROC AUC）：平均得分: {cv_scores.mean():.4f} (±{cv_scores.std():.4f})")
    
    # 4. 网格搜索
    grid_search = GridSearchCV(
        estimator=RandomForestClassifier(**{k: v for k, v in CONFIG['base_params'].items() 
                                          if k not in CONFIG['param_grid']}),
        param_grid=CONFIG['param_grid'],
        cv=5, scoring='roc_auc', n_jobs=-1, verbose=1
    )
    grid_search.fit(X_train, y_train)
    best_rf = grid_search.best_estimator_
    
    print(f"\n最佳参数: {grid_search.best_params_}")
    print(f"最佳交叉验证ROC AUC: {grid_search.best_score_:.4f}")
    
    # 5. 模型训练和预测
    best_rf.fit(X_train, y_train)
    y_train_prob = best_rf.predict_proba(X_train)[:, 1]
    y_prob = best_rf.predict_proba(X_test)[:, 1]
    
    # 6. 阈值优化
    optimal_threshold_f1 = find_optimal_threshold(y_train, y_train_prob, 'f1')
    y_pred_optimal = (y_prob >= optimal_threshold_f1).astype(int)
    optimal_threshold_youden = find_optimal_threshold(y_train, y_train_prob, 'youden')
    
    # 7. 计算评估指标
    test_roc_auc = roc_auc_score(y_test, y_prob)
    test_pr_auc = average_precision_score(y_test, y_prob)
    
    # 8. 可视化
    save_individual_plots(best_rf, X_test, y_test, y_prob, X_train, y_train, 
                         y_train_prob, optimal_threshold_f1, optimal_threshold_youden, 
                         test_roc_auc, test_pr_auc)
    
    # 9. 保存模型和必要数据
    os.makedirs(CONFIG['output_dir'], exist_ok=True)
    dump(best_rf, f"{CONFIG['output_dir']}/best_rf_model.joblib")
    dump(X_train, f"{CONFIG['output_dir']}/X_train.joblib")
    dump(X_test, f"{CONFIG['output_dir']}/X_test.joblib")
    dump(y_train, f"{CONFIG['output_dir']}/y_train.joblib")
    dump(y_test, f"{CONFIG['output_dir']}/y_test.joblib")
    dump(y_prob, f"{CONFIG['output_dir']}/y_prob.joblib")
    
    # 10. 汇总结果并导出 Excel
    excel_path = os.path.join(CONFIG['output_dir'], "RF_results.xlsx")

    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # 10.1 交叉验证结果
        cv_df = pd.DataFrame({
            "fold": np.arange(1, len(cv_scores)+1),
            "roc_auc": cv_scores
        })
        cv_df.loc["mean"] = ["mean", cv_scores.mean()]
        cv_df.loc["std"]  = ["std",  cv_scores.std()]
        cv_df.to_excel(writer, sheet_name="CV_result", index=False)

        # 10.2 最佳参数
        best_params_df = pd.Series(grid_search.best_params_).to_frame("value")
        best_params_df.to_excel(writer, sheet_name="Best_params")

        # 10.3 训练集和测试集性能
        # 训练集性能
        y_train_pred_default = (y_train_prob >= 0.5).astype(int)
        train_metrics = pd.Series(
            calculate_metrics(y_train, y_train_pred_default, y_train_prob, prefix="train_")
        ).to_frame("value")
        
        # 测试集性能
        y_test_pred_default = (y_prob >= 0.5).astype(int)
        test_metrics = pd.Series(
            calculate_metrics(y_test, y_test_pred_default, y_prob, prefix="test_")
        ).to_frame("value")
        
        # 合并训练集和测试集性能
        performance_df = pd.concat([train_metrics, test_metrics], axis=1)
        performance_df.columns = ['train', 'test']
        performance_df.to_excel(writer, sheet_name="Performance_metrics")

        # 10.4 按 CKD 分组评估
        if "CKD" in data.columns:
            ckd_groups = data.loc[X_test.index, "CKD"]
            group_metrics = evaluate_by_ckd_group(best_rf, X_test, y_test,
                                                  ckd_groups,
                                                  threshold=0.5)
            group_df = pd.DataFrame(group_metrics).T
            group_df.index.name = "CKD_stage"
            group_df.to_excel(writer, sheet_name="Metrics_by_CKD")

        # 10.5 特征重要性
        feat_imp = pd.Series(best_rf.feature_importances_, index=X.columns)\
                    .sort_values(ascending=False)\
                    .to_frame("importance")
        feat_imp.to_excel(writer, sheet_name="Feature_importance")

    print(f"\n所有结果已汇总并保存至：{excel_path}")
    
    print("\n模型训练完成！模型和数据已保存，可以运行下一个cell进行SHAP分析。")

if __name__ == "__main__":
    main()

In [ ]:
# Cell 2: SHAP分析
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from joblib import dump, load
import shap

# ====================== 配置参数 ======================
CONFIG = {
    'output_dir': "re0/5_RF"
}

def _format_label(label: str) -> str:
    """统一格式化特征标签：下划线→空格；定量 2 位小数；定性整数"""
    label = label.replace('_', ' ')
    m = re.match(r'(.+?)\s*=\s*([\d\.eE\-\+]+)', label)
    if not m:
        return label
    name, val_str = m.groups()
    val = float(val_str)
    # 若标签以 =0 或 =1 结尾，视为定性变量
    if label.endswith(('=0', '=1')):
        val_fmt = f'{int(round(val))}'
    else:
        val_fmt = f'{val:.2f}'.rstrip('0').rstrip('.')
    return f'{name} = {val_fmt}'

def _save_single_force(exp_single, save_path):
    """保存单张力图"""
    fig = shap.plots.force(exp_single, matplotlib=True, show=False)
    ax = fig.axes[0]

    # 1. 收集并格式化文本
    base_val = float(exp_single.base_values)
    new_txts = []
    for t in ax.texts:
        txt = t.get_text().strip()
        if txt.startswith('f(x)'):                    # 去掉 f(x)
            continue
        elif txt.startswith('base value'):            # 重写 base value
            new_txts.append(('base', f'base value = {base_val:.2f}',
                             t.get_position(), t.get_fontsize(),
                             t.get_color(), t.get_horizontalalignment()))
        else:                                         # 普通特征
            new_txts.append(('feat', _format_label(txt),
                             t.get_position(), t.get_fontsize(),
                             t.get_color(), t.get_horizontalalignment()))

    # 2. 删除旧文本
    for t in list(ax.texts):
        t.remove()

    # 3. 重新添加
    for typ, txt, pos, fs, col, ha in new_txts:
        ax.text(*pos, txt, fontsize=fs, color=col, ha=ha, va='center')

    # 4. 微调坐标轴
    ax.set_ylabel('')
    ax.set_yticks([0, 1])
    ax.set_yticklabels(['lower', 'higher'])
    ax.yaxis.set_label_position('left')
    ax.yaxis.tick_left()

    x_min, x_max = ax.get_xlim()
    ax.set_xticks([round(x_min, 2), round(x_max, 2)])
    ax.set_xticklabels([f'{x_min:.2f}', f'{x_max:.2f}'])

    fig.tight_layout()
    fig.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close(fig)

def _save_shap_scatter_plots(exp, X_test, output_dir, max_features=20):
    """保存所有变量的SHAP值散点图"""
    print("正在生成SHAP散点图...")
    
    # 获取特征重要性排序
    mean_abs_shap = np.abs(exp.values).mean(axis=0)
    feature_importance_order = np.argsort(mean_abs_shap)[::-1]
    
    # 限制显示的特征数量
    display_features = min(max_features, len(X_test.columns))
    
    for i in range(display_features):
        feature_idx = feature_importance_order[i]
        feature_name = X_test.columns[feature_idx]
        
        # 创建散点图
        plt.figure(figsize=(10, 6))
        
        # 绘制SHAP值与特征值的散点图
        plt.scatter(X_test.iloc[:, feature_idx], exp.values[:, feature_idx], 
                   alpha=0.6, s=20, c='steelblue', edgecolors='none')
        
        # 添加标签和标题
        plt.xlabel(f'{feature_name} Value', fontsize=12)
        plt.ylabel(f'SHAP Value for {feature_name}', fontsize=12)
        plt.title(f'SHAP Dependence Plot: {feature_name}', fontsize=14)
        
        # 添加网格
        plt.grid(True, alpha=0.3)
        
        # 保存图像
        safe_feature_name = feature_name.replace('/', '_').replace('\\', '_').replace('*', '_')
        plt.tight_layout()
        plt.savefig(f"{output_dir}/shap_scatter_{safe_feature_name}.png", dpi=300, bbox_inches='tight')
        plt.close()
        
    print(f"已保存 {display_features} 个SHAP散点图")

def _save_shap_summary_plot(exp, output_dir, max_display=20):
    """保存SHAP摘要图（点图）"""
    print("正在生成SHAP摘要图...")
    
    # 创建摘要图
    plt.figure(figsize=(12, 8))
    shap.summary_plot(exp.values, exp.data, feature_names=exp.feature_names, 
                     max_display=max_display, show=False)
    plt.tight_layout()
    plt.savefig(f"{output_dir}/shap_summary_dot.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 创建另一种颜色的摘要图（可选）
    plt.figure(figsize=(12, 8))
    shap.summary_plot(exp.values, exp.data, feature_names=exp.feature_names, 
                     max_display=max_display, show=False, color='coolwarm')
    plt.tight_layout()
    plt.savefig(f"{output_dir}/shap_summary_dot_coolwarm.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    print("SHAP摘要图已保存")

def perform_shap_analysis():
    """执行SHAP分析"""
    print("\n开始 SHAP 分析...")
    
    # 加载模型和数据
    best_rf = load(f"{CONFIG['output_dir']}/best_rf_model.joblib")
    X_test = load(f"{CONFIG['output_dir']}/X_test.joblib")
    y_prob = load(f"{CONFIG['output_dir']}/y_prob.joblib")
    
    # 计算 SHAP 值
    explainer = shap.TreeExplainer(best_rf)
    shap_values = explainer(X_test)

    # 处理二分类情况
    if len(shap_values.values.shape) == 3:
        sv = shap_values.values[..., 1]
        base = shap_values.base_values[..., 1]
    else:
        sv = shap_values.values
        base = shap_values.base_values

    feature_names = [c.replace('_', ' ') for c in X_test.columns]

    exp = shap.Explanation(
        values=sv,
        base_values=base,
        data=X_test.values,
        feature_names=feature_names
    )

    # 条形图
    plt.figure()
    shap.plots.bar(exp, max_display=20, show=False)
    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/shap_summary_bar.png", dpi=300, bbox_inches='tight')
    plt.close()

    # 小提琴图
    plt.figure()
    shap.plots.violin(exp, max_display=20, show=False)
    plt.tight_layout()
    plt.savefig(f"{CONFIG['output_dir']}/shap_summary_violin.png", dpi=300, bbox_inches='tight')
    plt.close()
    
    # 新增：SHAP摘要图（点图）
    _save_shap_summary_plot(exp, CONFIG['output_dir'])
    
    # 新增：保存所有变量的SHAP值散点图
    _save_shap_scatter_plots(exp, X_test, CONFIG['output_dir'])

    # 力图（2 阴性 + 2 阳性）
    # 阴性（概率最低 2 个）
    neg_idx = y_prob.argsort()[:1]
    for k, idx in enumerate(neg_idx, 1):
        _save_single_force(exp[idx], f"{CONFIG['output_dir']}/shap_force_neg{k}.png")

    # 阳性（概率最高 2 个）
    pos_idx = y_prob.argsort()[-2:][::-1]
    for k, idx in enumerate(pos_idx, 1):
        _save_single_force(exp[idx], f"{CONFIG['output_dir']}/shap_force_pos{k}.png")
    
    # 将SHAP值添加到Excel文件中
    try:
        excel_path = os.path.join(CONFIG['output_dir'], "RF_results.xlsx")
        with pd.ExcelWriter(excel_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            # 提取SHAP值 - 修复维度问题
            if len(shap_values.values.shape) == 3:
                # 对于二分类问题，选择第二类（索引1）的SHAP值
                shap_array = shap_values.values[:, :, 1]
            else:
                shap_array = shap_values.values
            
            # 创建SHAP值DataFrame
            shap_df = pd.DataFrame(shap_array, 
                                 index=X_test.index, 
                                 columns=[f"SHAP_{col}" for col in X_test.columns])
            
            # 添加预测概率和真实标签
            y_test = load(f"{CONFIG['output_dir']}/y_test.joblib")
            shap_df['predicted_probability'] = y_prob
            shap_df['true_label'] = y_test.values
            shap_df.to_excel(writer, sheet_name="SHAP_values")

            # SHAP特征重要性（均值绝对值）
            if len(shap_values.values.shape) == 3:
                mean_abs_shap = np.abs(shap_values.values[:, :, 1]).mean(axis=0)
            else:
                mean_abs_shap = np.abs(shap_values.values).mean(axis=0)
            
            shap_importance = pd.DataFrame({
                'feature': X_test.columns,
                'mean_abs_shap': mean_abs_shap
            }).sort_values('mean_abs_shap', ascending=False)
            shap_importance.to_excel(writer, sheet_name="SHAP_importance", index=False)
            
        print(f"\nSHAP结果已添加到Excel文件：{excel_path}")
    except Exception as e:
        print(f"添加到Excel文件时出错：{e}")
    
    print("\nSHAP分析完成！")

if __name__ == "__main__":
    perform_shap_analysis()